In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gpt2email/transformers/default/1/config.json
/kaggle/input/gpt2email/transformers/default/1/merges.txt
/kaggle/input/gpt2email/transformers/default/1/training_args.bin
/kaggle/input/gpt2email/transformers/default/1/vocab.json
/kaggle/input/gpt2email/transformers/default/1/tokenizer_config.json
/kaggle/input/gpt2email/transformers/default/1/model.safetensors
/kaggle/input/gpt2email/transformers/default/1/special_tokens_map.json
/kaggle/input/gpt2email/transformers/default/1/generation_config.json
/kaggle/input/subjects/EmailSubjectTest.txt
/kaggle/input/subjects/EmailSubjectTrain.txt
/kaggle/input/bartfb_email/pytorch/default/1/config.json
/kaggle/input/bartfb_email/pytorch/default/1/merges.txt
/kaggle/input/bartfb_email/pytorch/default/1/training_args.bin
/kaggle/input/bartfb_email/pytorch/default/1/vocab.json
/kaggle/input/bartfb_email/pytorch/default/1/tokenizer_config.json
/kaggle/input/bartfb_email/pytorch/default/1/model.safetensors
/kaggle/input/bartfb_email/pytorch

In [2]:
!pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [3]:
 !pip install accelerate 

In [4]:
!pip install safetensors transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00a 0:00:010m


In [5]:
import os
import re
import nltk
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import evaluate
from safetensors.torch import load_file as load_safetensors
import string
from nltk.corpus import wordnet

2024-08-03 10:37:01.257841: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 10:37:01.257964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 10:37:01.385355: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Initialize the tokenizer and model

model_name = 'gpt2-large'  # You can use 'gpt2-medium', 'gpt2-large', 'gpt2-xl' for larger models

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


# Define a function to load weights correctly
model_path = "/kaggle/input/gpt2email/transformers/default/1"
state_dict = load_safetensors(f"{model_path}/model.safetensors")
model = GPT2LMHeadModel.from_pretrained(model_path, state_dict=state_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.safetensors:  82%|########1 | 2.65G/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

1

In [12]:
# Function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Reconstruct text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text


In [13]:
preprocess_text('hi how r u?')

'hi r u'

In [14]:
# Load and preprocess the test data from a .txt file
def load_and_preprocess_file(file_path):
    test_emails = []
    reference_subjects = []
    
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        # Read the entire file content
        content = file.read()
        
        # Split data based on '<end>' or any other separator you use
        entries = content.split("<end>")
        
        for entry in entries:
            email_match = re.search(r"Email : (.*?) <Sep>", entry, re.DOTALL)
            subject_match = re.search(r"Subject : (.*?) <Sep>", entry, re.DOTALL)
            
            if email_match and subject_match:
                email_body = email_match.group(1).strip()
                reference_subject = subject_match.group(1).strip()
                
                # Preprocess the email body
                preprocessed_email = preprocess_text(email_body)
                
                # Append preprocessed email and reference subject to lists
                test_emails.append(preprocessed_email)
                reference_subjects.append(reference_subject)
            else:
                print("No match found in entry:", entry)
    
    return test_emails, reference_subjects

In [15]:
# Path to your .txt file
file_path = '/kaggle/input/subjects/EmailSubjectTest.txt'

# Load and preprocess the data
test_emails, reference_subjects = load_and_preprocess_file(file_path)

# Print the results
print(f"Number of preprocessed test emails: {len(test_emails)}")
print(f"Number of reference subjects: {len(reference_subjects)}")

for email, subject in zip(test_emails, reference_subjects):
    print(f"Email: {email}")
    print(f"Reference Subject: {subject}")

No match found in entry: 

Number of preprocessed test emails: 1906
Number of reference subjects: 1906
Email: phillip could please favor would like read current title policy see say easement received copy closing dont know many page let know want handle getting copy made ill happy make copy whatever make easy thanks
Reference Subject: Huntley/question
Email: following report waiting approval day please review owner james w reitmeyer report name jreitmeyer day mgr queue
Reference Subject: Expense Reports Awaiting Your Approval
Email: last couple week started compile restartintegration plan netco far primarily focussed midback plan technology requirement driving factor several plan final stage completion including infrastructure jenny rub development jay webb enrononline webb forster hr david oxley cash management tom myers credit debbie brackett rest completed shortly need focus commercial plan slightly different focus john would like receive plan restartintegration plan january th orde

In [16]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9e291d807518c025692466f4ca94a5cdb03b6d10f11721e369bec5fc93142496
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [17]:
import torch
import evaluate

# Function to generate the first four words of an email text
def get_first_four_words_split(text):
    words = text.split()
    return ' '.join(words[:4])

# Function to generate subject lines
def generate_subject(model, tokenizer, email_text, max_length=512):
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        input_ids = tokenizer.encode(email_text, return_tensors='pt').to(device)
        
        # Debugging print statements
        print(f"Generating subject for email: {email_text[:100]}...")  # Print part of email text
        print(f"Input IDs shape: {input_ids.shape}")
        
        with torch.no_grad():
            output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
        
        generated_subject = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Optionally, prepend the first four words to the generated subject
        first_four_words = get_first_four_words_split(email_text)
        final_subject = f"{first_four_words} - {generated_subject}"
        
        return final_subject
    except Exception as e:
        print(f"Error generating subject: {e}")
        raise

# Example list of test emails
test_emails = [
    "Hello team, we need to discuss the upcoming project deadlines and deliverables.",
    "Reminder: Our meeting is scheduled for tomorrow at 10 AM.",
    "Please review the attached document and provide your feedback."
]

# Example list of reference subjects
reference_subjects = [
    "Project deadlines and deliverables discussion",
    "Meeting scheduled for tomorrow at 10 AM",
    "Document review and feedback request"
]

# Initialize model and tokenizer (replace with actual loading code)
# model = ...
# tokenizer = ...

# Generate subject lines for test emails
generated_subjects = []
try:
    generated_subjects = [generate_subject(model, tokenizer, email) for email in test_emails]
except Exception as e:
    print(f"Error generating subjects: {e}")
    raise

print("Generated subjects:")
for subject in generated_subjects:
    print(subject)

# Load evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# Prepare references for BLEU and ROUGE evaluation
# BLEU expects a list of lists of references for each prediction
references = [[ref] for ref in reference_subjects]

# Evaluate BLEU and ROUGE scores
try:
    bleu_results = bleu_metric.compute(predictions=generated_subjects, references=references)
    rouge_results = rouge_metric.compute(predictions=generated_subjects, references=reference_subjects)
    
    print("BLEU score:", bleu_results)
    print("ROUGE scores:", rouge_results)
except Exception as e:
    print(f"Error evaluating metrics: {e}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating subject for email: Hello team, we need to discuss the upcoming project deadlines and deliverables....
Input IDs shape: torch.Size([1, 15])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating subject for email: Reminder: Our meeting is scheduled for tomorrow at 10 AM....
Input IDs shape: torch.Size([1, 13])
Generating subject for email: Please review the attached document and provide your feedback....
Input IDs shape: torch.Size([1, 10])
Generated subjects:
Hello team, we need - Hello team, we need to discuss the upcoming project deadlines and deliverables.
Reminder: Our meeting is - Reminder: Our meeting is scheduled for tomorrow at 10 AM.
Please review the attached - Please review the attached document and provide your feedback.


BLEU score: {'bleu': 0.12313840791270557, 'precisions': [0.22641509433962265, 0.14, 0.10638297872340426, 0.06818181818181818], 'brevity_penalty': 1.0, 'length_ratio': 3.1176470588235294, 'translation_length': 53, 'reference_length': 17}
ROUGE scores: {'rouge1': 0.4973544973544974, 'rouge2': 0.2807017543859649, 'rougeL': 0.4603174603174603, 'rougeLsum': 0.4603174603174603}


In [ ]:
!pip install --upgrade torch torchvision torchaudio evaluate